In [1]:
import os, sys,shutil, random,math,glob
import girder_client as gc
from pprint import pprint
from IPython.display import Image
from pprint import pprint
from random import shuffle
sys.path.append("../helperLibs")

import DSAHelperFunctions as DSA
import config as c
API_URL = "http://candygram.neurology.emory.edu:8080/api/v1"
dsaGC = gc.GirderClient(apiUrl=API_URL)

In [2]:
dsaGC.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [31]:
## Provide some info about the data set I am generating
testSetSpec = dict(
    testSetName = "TCGA_MultiClass.V1",
    testSetDescription = """Set contains DX1 slides; the data set is split into 80/20  training/testing
    By Default, it will assume a 256x256 padded thumbnail image output although you can change this if you want
    in the download script""",
    defaultMacroSize = 256,
    setSource = "TCGA",
    trainRatio = 0.80,
    testRatio= 0.20,
    valRatio = 0.00
)

In [3]:
### Go to girder and get the list of images
##going to use some of the special tcga endpoints to make my life a little easier
## This gets the cancer types and/or cohorts we have data for
TCGACohortList = dsaGC.get('/tcga/cohort')
cohortData = {}
for x in  TCGACohortList['data']:
    cohortData[x['lowerName']] = { '_id': x['_id'], 'lowerName': x['lowerName']}

In [7]:
### We just got all of the cohorts, all we care about for now is the BRCA cohort
### Let's get all of the DX1 images for our initial test..
## get the cases for the cohort, but ONLY am getting the first DX1 slide...
dxSlidesFound = totalSlidesFound =  0

for cohort in cohortData:
    slideList = dsaGC.get('/tcga/cohort/%s/images?&limit=9000' % cohortData[cohort]['_id'])
    # ## I only want to get images that end in DX1 for now

    dxSlidesForCohort = []
    for sld in slideList['data']:
        totalSlidesFound +=1
        if 'DX1' in sld['tcga']['barcode']:
            dxSlidesFound +=1
            dxSlidesForCohort.append(sld)
        else:
            ## In the future may want to dump all of the other slides and use for additional testing
            pass
        DSA.LinePrinter("You have processed %d slides and %d were DX slides" % (totalSlidesFound,dxSlidesFound))
    
    print "\nYou have found a total of %d slides for cohort %s" % ( len(dxSlidesForCohort),cohort)
        
    cohortData[cohort]['dxCaseList'] = dxSlidesForCohort

You have processed 396 slides and 156 were DX slidesYou have foudn a total of 156 slides for cohort esca
You have processed 506 slides and 207 were DX slidesYou have foudn a total of 51 slides for cohort dlbc
You have processed 664 slides and 268 were DX slidesYou have foudn a total of 61 slides for cohort ucs
You have processed 982 slides and 389 were DX slidesYou have foudn a total of 121 slides for cohort thym
You have processed 2680 slides and 893 were DX slidesYou have foudn a total of 504 slides for cohort lusc
You have processed 3928 slides and 1411 were DX slidesYou have foudn a total of 518 slides for cohort thca
You have processed 4934 slides and 1800 were DX slidesYou have foudn a total of 389 slides for cohort blca
You have processed 6475 slides and 2333 were DX slidesYou have foudn a total of 533 slides for cohort ucec
You have processed 6632 slides and 2413 were DX slidesYou have foudn a total of 80 slides for cohort uvm
You have processed 7288 slides and 2684 were DX sli

In [25]:
## Generate the training and test Sets for all the cohorts... could potentially ignore cohorts if they
## Don't have enough images
combinedTrainSet = {}
combinedTestSet = {}
combinedValSet = {}

for c in cohortData:
    cohortSamples = cohortData[c]['dxCaseList']
    print c, len(cohortSamples)
    cohortTrain,cohortVal,cohortTest = generateTrainTestVal(cohortSamples)
    combinedTrainSet[c] = cohortTrain
    combinedTestSet[c]  = cohortTest
    combinedValSet[c] = cohortVal

esca 156
dlbc 51
ucs 61
thym 121
lusc 504
thca 518
blca 389
ucec 533
uvm 80
cesc 271
sarc 255
kirp 287
prad 427
skcm 459
kirc 531
kich 121
luad 510
tgct 154
read 169
lgg 1064
stad 459
meso 80
coad 475
pcpg 180
acc 56
gbm 1143
chol 39
hnsc 480
paad 195
brca 1438
ov 106
lihc 365
32


In [ ]:
# ### Do some quick sanity checks and see how many potential slides each cohort has.. if a cohort only has < 100
# ### Slides we probably dont' want to include it in our initial model(s)
#     train_ratio = testSetSpec['train_ratio']
#     val_ratio = testSetSpec['val_ratio']
#     test_ratio = testSetSpec['test_ratio']



In [28]:
def remove_extra_keys( sampleSet, keysToRemove='default' ):
    ### There's a lot of extra metadata for each item that is largely irrelevant, so this will for now
    ### use a blacklist to remove keys I know are not useful
    if (keysToRemove=='default'):
        fieldsToPrune = ['description','copyOfItem','folderId','baseParentId','baseParentType','created','creatorId','largeImage','updated','lowerName','size']
    ### This is a default set of keys I have not found very helpful..
    for s in sampleSet:
        for f in fieldsToPrune:
            if f in s:
                del s[f]        

In [29]:
## Define the sample Set, then shuffle it-- 


## In this case we have no validation data because Keras automagically does it for us

def generateTrainTestVal( samples, trainRatio=0.8, testRatio=0.2,valRatio=0,removeExtraKeys = True ):
    ### Given a list, this will split it into train,test and validation spots
    # This returns a tuple of trainSet, valSet, testSet
    shuffle(samples)

    ## Split up the original shuffled samples into 3 chunks of diff lengths based on proportions above..
    ## Remove extra keys that I don't want to output
    if removeExtraKeys:
        remove_extra_keys(samples)

    N = len(samples)
    endTrainingIndex = int(trainRatio*N)
    endValidationIndex = int(valRatio*N) + endTrainingIndex
    startTestSetIndex = endValidationIndex  ## or -1 or something else..

    trainSet = samples[0:endTrainingIndex]
    valSet = samples[endTrainingIndex:endValidationIndex]
    testSet = samples[startTestSetIndex:]
    return (trainSet,valSet,testSet)

In [20]:
generateTrainTestVal( cohortData['gbm']['dxCaseList'])

{'testSet': [{u'_id': u'597ddda492ca9a000de47fd6',
   u'name': u'TCGA-27-1835-01Z-00-DX1.e391c3a2-eb2a-4c3d-94be-16951be30dab.svs',
   u'tcga': {u'barcode': u'TCGA-27-1835-01Z-00-DX1',
    u'case': u'TCGA-27-1835',
    u'caseId': u'597e230192ca9a000de59f6f',
    u'cohort': u'gbm',
    u'label': u'TCGA-27-1835',
    u'type': u'image',
    u'uuid': u'e391c3a2-eb2a-4c3d-94be-16951be30dab'}},
  {u'_id': u'5ae22c1b92ca9a00174b9711',
   u'name': u'TCGA-06-0221-01Z-00-DX1.bc444e2a-9530-453a-b494-ec31ee5db9c1.svs',
   u'tcga': {u'barcode': u'TCGA-06-0221-01Z-00-DX1',
    u'case': u'TCGA-06-0221',
    u'caseId': u'597e22d492ca9a000de59924',
    u'cohort': u'gbm',
    u'label': u'TCGA-06-0221',
    u'type': u'image',
    u'uuid': u'bc444e2a-9530-453a-b494-ec31ee5db9c1'}},
  {u'_id': u'5ae22c4192ca9a00174ba5bc',
   u'name': u'TCGA-19-2631-01Z-00-DX1.cfe16b12-cd2e-4c77-927d-fa3c7ff54e11.svs',
   u'tcga': {u'barcode': u'TCGA-19-2631-01Z-00-DX1',
    u'case': u'TCGA-19-2631',
    u'caseId': u'597e22

In [35]:

testSetSpec['cohortLabels'] = combinedTrainSet.keys() 
dataSetDefintion = { "meta": testSetSpec, "serverAPIUrl":API_URL , "valSet": combinedValSet, "trainingSet": combinedTrainSet, "testSet": combinedTestSet}

In [36]:
import json
with open("TCGA.MultiClass.MacroImageSet.json","w") as fp:
    json.dump(dataSetDefintion,fp)

In [1]:
dataSetDefintion.keys()

NameError: name 'dataSetDefintion' is not defined

In [ ]:
cancerSubtypes = os.listdir(thumbBaseDir)
print len(cancerSubtypes),"cancer subtypes have thumbnail Data"

cancerDataThumbDict = {}

for c in cancerSubtypes:
    cancerDataThumbDict[c] = {}
    currentThumbRootDir = os.path.join(thumbBaseDir,c)
    cancerDataThumbDict[c]['rootDir'] = currentThumbRootDir
    
    cancerDataThumbDict[c]['jpgSize'] = "256x256"
    cancerDataThumbDict[c]['jpgThumbList'] = [x for x in os.listdir(currentThumbRootDir) if x.endswith(".jpg")]

In [ ]:
for c in cancerDataThumbDict:
    print c, len(cancerDataThumbDict[c]['jpgThumbList'])
    
del cancerDataThumbDict['pcpg']

In [ ]:
## CREATE TESTING VALIDATION AND TRAINING SETS AND COPY OVER THE IMAGES TO THESE DIRECTORIES
trainProportion = 0.75
testingProportion = 0.2
validationProportion =0.05

from random import shuffle

traingTestingVal_OutputDir
for c in cancerDataThumbDict:
    #print cancerDataThumbDict[c]
    curCancerThumbList = cancerDataThumbDict[c]['jpgThumbList']
    
    
    
    curCancerThumbList = shuffle(curCancerThumbList)

    
    imgsInCohort = len(cancerDataThumbDict[c]['jpgThumbList'])
    for idx,img in enumerate(cancerDataThumbDict[c]['jpgThumbList']):
        
        ### Create Training Set
        trainingOutputDir = os.path.join(traingTestingVal_OutputDir,"train",c)
        testingOutputDir = os.path.join(traingTestingVal_OutputDir,"test",c)
        validationOutputDir = os.path.join(traingTestingVal_OutputDir,"val",c)
        
        if not os.path.isdir(trainingOutputDir):
            os.makedirs(trainingOutputDir)
        if not os.path.isdir(testingOutputDir):
            os.makedirs(testingOutputDir)
        if not os.path.isdir(validationOutputDir):
            os.makedirs(validationOutputDir)
        
        curProp = idx/float(imgsInCohort)
        
        if curProp < trainProportion:
            shutil.copy( os.path.join(cancerDataThumbDict[c]['rootDir'],img), trainingOutputDir)
        elif (curProp >= trainProportion and curProp <= (trainProportion+validationProportion)  ):
            shutil.copy( os.path.join(cancerDataThumbDict[c]['rootDir'],img), validationOutputDir)
        else:
            shutil.copy( os.path.join(cancerDataThumbDict[c]['rootDir'],img), testingOutputDir)